# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [37]:
from openai import OpenAI
import base64
import requests
import json

# 初始化 OpenAI 客户端
client = OpenAI(api_key='sk-k9jM0trGFbAQm1cu6bD951Cd503c4203A64d6aFf7aD8Ff09',base_url='https://api.xiaoai.plus/v1') 

def query_base64_image_description(image_path, prompt="图里的内容讲了什么？你能对它进行分析吗？", history_message = None, last_reply = None, max_tokens=1000 ):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }
    # message中的content
    content = []

    # 如果传入image path不为空时才对image进行操作
    if image_path == "":
        content = [{"type": "text", "text": prompt}]
    else :
        # 获取图像的 Base64 编码字符串
        base64_image = encode_image(image_path)
        content = [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
        ]
    messages = [{"role": "user", "content": content}]
    # 当history_message 和last_reply不为空时将history_message 和last_reply进行组装以支持历史对话
    if history_message != None and last_reply!= None:
        message = {'role': 'assistant', 'content': last_reply}
        history_message.append(message)
        history_message.append(messages[0])
        messages = history_message
    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": messages,
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return {
            'messages': messages,
            'current_reply':content
        }
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图展示了1980年到2020年间，美国、中国、日本和德国的国内生产总值（GDP）比较。每个国家的GDP以万亿美元为单位标出。从图中可以看出：

- **美国（蓝线）**：GDP持续增长，从1980年的约3万亿美金增长到2020年的超过20万亿美金。
- **中国（红线）**：自1980年起，GDP增长非常显著，从不到1万亿美金增长至接近15万亿美金。
- **日本（紫线）**：1980年至1995年GDP增长明显，之后增长放缓，并在2005年至2015年间保持相对平稳。
- **德国（绿线）**：GDP增长较为平稳，从1980年的不到1万亿美金增长至2020年的约4万亿美金。

总体而言，图表清晰地表示了这四个经济体在过去四十年间的经济增长情况，其中中国的增长尤其引人注目，显示了其经济的迅速崛起。美国则保持了其全球经济领导地位的增长趋势。日本在90年代后增长放缓，而德国则显示出稳定的增长模式。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [10]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [11]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

![](./images/gpt-4v.jpg)

In [42]:

data = query_base64_image_description("./images/gpt-4v.jpg")
print(data['current_reply'])


这张图展示了NLP（自然语言处理）领域中的一些重要发展阶段及相关技术。具体内容如下：

1. **预训练在CV（计算机视觉）和NLP中的应用**：
   - **Frozen**：预训练模型参数固定，不进行调整。
   - **Fine-tuning**：在预训练模型的基础上进行微调，使其更适应特定任务。

2. **神经网络表示模型（NN LM）**：
   - **word2vec**：一种词向量表示方法，包括**CBOW（Continuous Bag of Words）**和**skip-gram**两种模型。
   - **GloVe（Global Vectors for Word Representation）**：另一种词向量表示方法。

3. **ELMO（基于上下文的词嵌入）**：
   - **双向双向LSTM**：使用双向LSTM网络生成词嵌入。
   - **动态调整embedding**：根据上下文动态调整词的嵌入表示。

4. **GPT（基于Transformer模型的生成预训练模型）**：
   - 利用Transformer模型进行预训练，替代传统的LSTM。
   - **Fine-tuning**：对预训练的GPT模型进行微调。

5. **Bert**：尽管图中未详细展开，但BERT（Bidirectional Encoder Representations from Transformers）也是一个基于Transformer的预训练模型，注重双向的上下文信息。

图中的内容按时间顺序展示了NLP领域中预训练模型的演变过程，从最初的静态词向量表示（如word2vec、GloVe）到上下文相关的词嵌入（如ELMO），再到基于Transformer的生成预训练模型（如GPT），最后是双向编码器表示模型（如BERT）。

每个阶段都包含了对应的技术细节，例如使用的神经网络结构、预训练方法及微调策略等。这些信息有助于理解NLP模型的发展趋势及其在实际应用中的改进点。


In [43]:
from IPython.display import display, Markdown
# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(data['current_reply']))

这张图展示了NLP（自然语言处理）领域中的一些重要发展阶段及相关技术。具体内容如下：

1. **预训练在CV（计算机视觉）和NLP中的应用**：
   - **Frozen**：预训练模型参数固定，不进行调整。
   - **Fine-tuning**：在预训练模型的基础上进行微调，使其更适应特定任务。

2. **神经网络表示模型（NN LM）**：
   - **word2vec**：一种词向量表示方法，包括**CBOW（Continuous Bag of Words）**和**skip-gram**两种模型。
   - **GloVe（Global Vectors for Word Representation）**：另一种词向量表示方法。

3. **ELMO（基于上下文的词嵌入）**：
   - **双向双向LSTM**：使用双向LSTM网络生成词嵌入。
   - **动态调整embedding**：根据上下文动态调整词的嵌入表示。

4. **GPT（基于Transformer模型的生成预训练模型）**：
   - 利用Transformer模型进行预训练，替代传统的LSTM。
   - **Fine-tuning**：对预训练的GPT模型进行微调。

5. **Bert**：尽管图中未详细展开，但BERT（Bidirectional Encoder Representations from Transformers）也是一个基于Transformer的预训练模型，注重双向的上下文信息。

图中的内容按时间顺序展示了NLP领域中预训练模型的演变过程，从最初的静态词向量表示（如word2vec、GloVe）到上下文相关的词嵌入（如ELMO），再到基于Transformer的生成预训练模型（如GPT），最后是双向编码器表示模型（如BERT）。

每个阶段都包含了对应的技术细节，例如使用的神经网络结构、预训练方法及微调策略等。这些信息有助于理解NLP模型的发展趋势及其在实际应用中的改进点。

In [44]:
data = query_base64_image_description("","对于上述的分析，在内容上你觉得还有那些不足之处？请进行适当补充",data['messages'], data['current_reply'])
display(Markdown(data['current_reply']))

对于图文中的分析，可以再进一步补充和阐述一些细节，以加深对各种技术的理解：

1. **预训练模型的比较**：
   - 对于“Frozen”和“Fine-tuning”，可进一步解释其在实际应用中的差异与选择依据。例如，完全冻结预训练权重常用于迁移学习的场景，而微调则适用于任务与预训练数据相似度较高的情形。

2. **神经网络表示模型深入**：
   - **word2vec**的**CBOW**与**skip-gram**模型的具体差异和适用场景可以更详细说明。CBOW是预测目标词基于上下文，而skip-gram则是通过目标词来预测上下文，后者通常对小数据集更有效。
   - **GloVe**模型的工作原理和与word2vec的区别可进一步阐释，如GloVe是通过全局词频统计信息来生成词向量。

3. **ELMO的额外细节**：
   - 补充ELMO如何利用双向LSTM捕捉复杂上下文关系，并具体说明其如何实现词义的动态调整。

4. **GPT和BERT的发展及差异**：
   - 对于GPT，可以更明确地介绍其使用单向Transformer结构的原因及其在自然语言理解和生成中的应用。
   - BERT的训练技术（如Masked Language Model和Next Sentence Prediction），以及其与GPT在设计哲学（双向 vs. 单向）上的本质区别应更详细说明。
   - 也可以补充这些模型对下游任务的影响，如提升了哪些特定NLP任务的性能。

5. **技术要点的适应性和限制**：
   - 分析每种技术方法的优势与局限，例如BERT的高内存消耗和训练成本，以及是否有简化或改进的模型（如DistilBERT, ALBERT等）。

通过这样的补充，可以为读者或学习者提供一个更全面、更深入的技术视角，并帮助他们更好地理解每种模型的特点、用途及其在NLP领域的应用背景。

In [45]:
data = query_base64_image_description("","对于你提到的DistilBERT，我很感兴趣，可以为我详细讲讲吗？",data['messages'], data['current_reply'])
display(Markdown(data['current_reply']))

当然，DistilBERT 是一个精简化的BERT模型，旨在减少模型的大小并提高运行速度，同时保持较高的性能。这种模型由Hugging Face团队提出，是对原始BERT模型的一个重要的缩减和优化。

### 核心特征

1. **模型大小和速度**：
   - DistilBERT的模型大小是原始BERT模型的一半，这意味着它的参数数量大约减少了40%至60%，但据报告能保持97%的BERT性能。
   - 这种减少的模型复杂性允许DistilBERT比BERT更快地进行训练和预测，增强了其在资源受限的环境下的适用性。

2. **知识蒸馏**：
   - DistilBERT采用了一种称为“知识蒸馏”的技术。在这个过程中，一个小型的“学生”模型学习模仿一个大型的“教师”模型（在本案例中，BERT是教师）。通过这种方式，小模型能够学习到大模型的行为和概念。
   - 蒸馏过程主要涉及训练学生模型对教师模型的输出进行模仿，这不仅仅是匹配最终输出，还包括中间层的输出，以充分利用教师模型的内部知识。

3. **训练方法**：
   - 除了标准的蒸馏损失，DistilBERT的训练还结合了交叉熵损失，这有助于学生直接从真实标签学习。
   - 还加入了对教师模型中间层的正则化，以更好地模拟教师模型的内部行为。

### 应用场景

DistilBERT适用于需要快速响应或资源有限的环境，如移动设备和在线服务，可以用于文本分类、问答系统、自然语言理解任务等，典型的使用场景包括：

- 在用户交互需快速响应的应用中，如聊天机器人和个人助理。
- 在计算资源有限的设备上，如手机和嵌入式系统中。
- 在需要处理大量数据的后端系统中，以减少处理时间和成本。

### 总结

DistilBERT提供了一种高效的方式来利用BERT的强大能力，同时显著降低了资源需求和运行成本。这使得在多种场合下能够更实用和可行，尤其是在对模型大小和推理速度有严格要求的应用中。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。